BL.EN.U4CSE21189 ; Siddhant Ashwani ; CSE - C

In [1]:
import pandas as pd
from collections import defaultdict
from scipy.stats import chi2_contingency
from itertools import product   # Using itertools to go through the dataset
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB  # Gaussian Naive Bayes is great for numeric input. Applying Ordinal encoding for numeric conversion.
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd  # Importing pandas for data handling

A1. For the data provided below, calculate the prior probability for each class.

In [2]:
#prior probability = probability of an event before it is collected


In [6]:
# Data samples
new_samples = [
    {'age': '<=30', 'income': 'high', 'student': 'no', 'credit_rating': 'fair', 'purchase_decision': 'no'},
    {'age': '<=30', 'income': 'high', 'student': 'no', 'credit_rating': 'excellent', 'purchase_decision': 'no'},
    {'age': '31..40', 'income': 'high', 'student': 'no', 'credit_rating': 'fair', 'purchase_decision': 'yes'},
    {'age': '>40', 'income': 'medium', 'student': 'no', 'credit_rating': 'fair', 'purchase_decision': 'yes'},
    {'age': '>40', 'income': 'low', 'student': 'yes', 'credit_rating': 'fair', 'purchase_decision': 'yes'},
    {'age': '>40', 'income': 'low', 'student': 'yes', 'credit_rating': 'excellent', 'purchase_decision': 'no'},
    {'age': '31..40', 'income': 'low', 'student': 'yes', 'credit_rating': 'excellent', 'purchase_decision': 'yes'},
    {'age': '<=30', 'income': 'medium', 'student': 'no', 'credit_rating': 'fair', 'purchase_decision': 'no'},
    {'age': '<=30', 'income': 'low', 'student': 'yes', 'credit_rating': 'fair', 'purchase_decision': 'yes'},
    {'age': '>40', 'income': 'medium', 'student': 'yes', 'credit_rating': 'fair', 'purchase_decision': 'yes'},
    {'age': '<=30', 'income': 'medium', 'student': 'yes', 'credit_rating': 'excellent', 'purchase_decision': 'yes'},
    {'age': '31..40', 'income': 'high', 'student': 'no', 'credit_rating': 'excellent', 'purchase_decision': 'yes'},
    {'age': '31..40', 'income': 'medium', 'student': 'yes', 'credit_rating': 'fair', 'purchase_decision': 'no'},
    {'age': '>40', 'income': 'high', 'student': 'no', 'credit_rating': 'excellent', 'purchase_decision': 'no'}
]

# Calculate the total number of instances
total_new_samples = len(new_samples)

# Determine the prior probabilities for each decision
unique_decisions = set(sample['purchase_decision'] for sample in new_samples)
prior_probs = {}

for decision_label in unique_decisions:
    decision_count = sum(1 for sample in new_samples if sample['purchase_decision'] == decision_label)
    prior_probs[decision_label] = decision_count / total_new_samples

print("Prior Probabilities for New Data:")
for decision_label, prior_prob in prior_probs.items():
    print(f"P({decision_label}) = {prior_prob:.4f}")


Prior Probabilities for New Data:
P(yes) = 0.5714
P(no) = 0.4286


A2.Calculate the class conditional densities for various features & classes. Observe if any class conditional density has zero values.

In [7]:
# Initialize dictionaries to store class-specific conditional probabilities
class_conditional_probs_updated = defaultdict(dict)  # defaultdict, a dictionary-like object, is employed here

# Calculate conditional probabilities for each feature and class
for feature_label in new_samples[0].keys():  # Assuming features are in the first dictionary
    if feature_label != 'purchase_decision':  # Exclude the class label
        for decision_label_updated in unique_decisions:
            # Filter instances with the given feature and decision label
            feature_decision_instances = [sample_updated for sample_updated in new_samples if sample_updated['purchase_decision'] == decision_label_updated and sample_updated[feature_label] == sample_updated[feature_label]]

            # Calculate class conditional probability
            probability_updated = len(feature_decision_instances) / prior_probs[decision_label_updated]
            class_conditional_probs_updated[decision_label_updated][feature_label] = probability_updated

# Display the class conditional probabilities
print("Class Conditional Probabilities (Updated):")
for decision_label_updated, probabilities_updated in class_conditional_probs_updated.items():
    print(f"For decision {decision_label_updated}:")
    for feature_label, probability_updated in probabilities_updated.items():
        print(f"P({feature_label} | {decision_label_updated}) = {probability_updated:.4f}")

# Check for any class conditional probability with zero values
zero_prob_features_updated = [(decision_label_updated, feature_label) for decision_label_updated, probabilities_updated in class_conditional_probs_updated.items() for feature_label, probability_updated in probabilities_updated.items() if probability_updated == 0]

if zero_prob_features_updated:
    print("\nFeatures with zero class conditional probabilities (Updated):")
    for decision_label_updated, feature_label in zero_prob_features_updated:
        print(f"For decision {decision_label_updated}, feature {feature_label} has zero probability.")
else:
    print("\nNo features have zero class conditional probabilities (Updated).")


Class Conditional Probabilities (Updated):
For decision yes:
P(age | yes) = 14.0000
P(income | yes) = 14.0000
P(student | yes) = 14.0000
P(credit_rating | yes) = 14.0000
For decision no:
P(age | no) = 14.0000
P(income | no) = 14.0000
P(student | no) = 14.0000
P(credit_rating | no) = 14.0000

No features have zero class conditional probabilities (Updated).


A3. Test for independence between the 4 given features.

In [8]:
# Chi-squared test for independence - Null hypothesis: The two variables are independent

# Extract features and labels from the updated dataset
age_vals = [sample_updated['age'] for sample_updated in new_samples]
income_vals = [sample_updated['income'] for sample_updated in new_samples]
student_status = [sample_updated['student'] for sample_updated in new_samples]
credit_rating_vals = [sample_updated['credit_rating'] for sample_updated in new_samples]
purchase_decision = [sample_updated['purchase_decision'] for sample_updated in new_samples]

# Create a set of unique values for each feature
unique_vals = {
    'age': set(age_vals),
    'income': set(income_vals),
    'student': set(student_status),
    'credit_rating': set(credit_rating_vals),
    'purchase_decision': set(purchase_decision)
}

# Generate all possible combinations of values for the features
combinations_updated = list(product(unique_vals['age'], unique_vals['income'], unique_vals['student'], unique_vals['credit_rating'], unique_vals['purchase_decision']))

# Construct a contingency table with Laplace smoothing
contingency_table_updated = []
epsilon_updated = 1e-10  # small constant to prevent zero expected frequencies

for age_val, inc_val, stud_val, credit_val, decision_val in combinations_updated:
    count_updated = sum(1 for i in range(total_new_samples) if age_vals[i] == age_val and purchase_decision[i] == decision_val and income_vals[i] == inc_val and student_status[i] == stud_val and credit_rating_vals[i] == credit_val)
    contingency_table_updated.append([count_updated + epsilon_updated])

# Perform Chi-squared test for independence
chi2_updated, p_updated, _, _ = chi2_contingency(contingency_table_updated)

# Display the test results
print(f"Chi-squared value (Updated): {chi2_updated}")
print(f"P-value (Updated): {p_updated}")

# Set the significance level (e.g., alpha = 0.05)
alpha_updated = 0.05    # alpha can be set to any value between 0 and 1
print("\nTest Result (Updated):")
if p_updated < alpha_updated:
    print("Reject the null hypothesis. Features are not independent.")
else:
    print("Fail to reject the null hypothesis. Features are independent.")


Chi-squared value (Updated): 0.0
P-value (Updated): 1.0

Test Result (Updated):
Fail to reject the null hypothesis. Features are independent.


A4. Build a Naïve-Bayes (NB) classifier for the above given data.

In [9]:
# Extract features and target variable for the updated dataset
data_features_updated = np.array([[sample_updated['age'], sample_updated['income'], sample_updated['student'], sample_updated['credit_rating']] for sample_updated in new_samples])
target_variable_updated = np.array([sample_updated['purchase_decision'] for sample_updated in new_samples])

# Utilize LabelEncoder to convert categorical data to numerical values
label_encoder_updated = LabelEncoder()
for i_updated in range(data_features_updated.shape[1]):
    data_features_updated[:, i_updated] = label_encoder_updated.fit_transform(data_features_updated[:, i_updated])

# Convert features to numeric values
data_features_updated = data_features_updated.astype(float)

# Split the data into training and testing sets
X_train_data_updated, X_test_data_updated, y_train_data_updated, y_test_data_updated = train_test_split(data_features_updated, target_variable_updated, test_size=0.2, random_state=42)

# Establish and train the Gaussian Naive Bayes classifier for the updated dataset
gnb_classifier_data_updated = GaussianNB()
gnb_classifier_data_updated.fit(X_train_data_updated, y_train_data_updated)

# Generate predictions on the test set for the updated dataset
predictions_data_updated = gnb_classifier_data_updated.predict(X_test_data_updated)

# Evaluate the classifier's performance for the updated dataset
accuracy_data_updated = np.sum(predictions_data_updated == y_test_data_updated) / len(y_test_data_updated)
print(f'Accuracy (Updated): {accuracy_data_updated:.2%}')


Accuracy (Updated): 66.67%


A5. Build a NB classifier for your own project data.

In [10]:
# Read data from Excel file for the new dataset
excel_data_updated = pd.read_excel('/content/embeddingsdata.xlsx')

# Extract features and target for the updated dataset
input_feats_updated = excel_data_updated.iloc[:, :-1]  # Features
output_target_updated = excel_data_updated.iloc[:, -1]      # Target data

# Utilize LabelEncoder to convert categorical data to numerical values
label_encoder_excel_updated = LabelEncoder()
for column_updated in input_feats_updated.columns:
    input_feats_updated[column_updated] = label_encoder_excel_updated.fit_transform(input_feats_updated[column_updated])

# Convert features to numeric values
input_feats_updated = input_feats_updated.astype(float)

# Split the data into training and testing sets for the updated dataset
train_feats_excel_updated, test_feats_excel_updated, train_target_excel_updated, test_target_excel_updated = train_test_split(input_feats_updated, output_target_updated, test_size=0.2, random_state=42)

# Establish and train the Gaussian Naive Bayes classifier for the updated dataset
gnb_classifier_excel_updated = GaussianNB()
gnb_classifier_excel_updated.fit(train_feats_excel_updated, train_target_excel_updated)

# Make predictions on the test set for the updated dataset
predictions_excel_updated = gnb_classifier_excel_updated.predict(test_feats_excel_updated)

# Evaluate the classifier's performance for the updated dataset
accuracy_excel_updated = np.sum(predictions_excel_updated == test_target_excel_updated) / len(test_target_excel_updated)
print(f'Accuracy (Updated): {accuracy_excel_updated:.2%}')


Accuracy (Updated): 55.00%
